In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import cm
import sys
import numpy as np
import pandas as pd 
import netCDF4 as nc
from cartopy import config
import cartopy.crs as ccrs
import os
import shutil
import tempfile
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [ ]:
np.set_printoptions(threshold=sys.maxsize)
os.chdir(r"C:\Users\eliza\Documents\Uni\Project\Data")

minlat = 110  # Set Latitude bounds (90 - 180 is NH, 0 - 90 is SH)
maxlat = 130  
minlong = 150  # Set Longitude Bounds
maxlong = 180  # 0-360 for checking latitudes


In [ ]:
fn = r"WS\1984.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]
jun = ds.variables['june'][:]
jun6hr = jun[0::2]
jun6hr = np.ma.filled(jun6hr, fill_value=0)
translated = np.roll(jun6hr, 180, axis=2)
junsize = np.size(jun6hr, 0)

In [ ]:
color_map = plt.cm.get_cmap('gist_ncar')
cmap = color_map.reversed()  # define the colormap

cmaplist = [cmap(i) for i in range(cmap.N)]
# force the first color entry to be grey
cmaplist[0] = "white"#(.5, .5, .5, 1.0)

# create the new map
cmap = mpl.colors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)
bounds = np.linspace(0, 11, 12)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

In [ ]:
a = -1
entries = []

for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1984\JUN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1984\JUN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = translated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = translated[a,j,i]
                    
        is_all_zero = np.all((WS == 0))
        
        if is_all_zero:
            pass
        else:
            fig = plt.figure(figsize=(20,20))
            gs = fig.add_gridspec(3, 3)
            ax1 = fig.add_subplot(gs[0:2, :], projection=ccrs.PlateCarree())
            ax1.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())
            ax1.gridlines(color='lightgrey', linestyle='-', draw_labels=True)
            plt.contourf(lon1, lat1, WS, transform=ccrs.PlateCarree(), cmap = cmap,norm=norm)
            ax1.coastlines()
            plt.colorbar()
            plt.show()
            print(a)
            entries.append(a)

In [ ]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1984\JUN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1984\JUN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        if a in entries:
            
            fig = plt.figure(figsize=(20,20))
            gs = fig.add_gridspec(3, 3)
            ax1 = fig.add_subplot(gs[0:2, :], projection=ccrs.PlateCarree())
            ax1.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())
            ax1.gridlines(color='lightgrey', linestyle='-', draw_labels=True)
            plt.contour(lon1, lat1, cdi, transform=ccrs.PlateCarree())
            ax1.coastlines()
            plt.show()
            print(a)


In [ ]:
for a in range(junsize):
    if a in entries:
        data = np.roll(jun6hr[a,:,:], 180, axis=0)
        fig = plt.figure(figsize=(20,20))
        gs = fig.add_gridspec(3, 3)
        ax1 = fig.add_subplot(gs[0:2, :], projection=ccrs.PlateCarree())
        ax1.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())
        ax1.gridlines(color='lightgrey', linestyle='-', draw_labels=True)
        plt.contourf(lon, lat, data.T, transform=ccrs.PlateCarree(), cmap = cmap,norm=norm)
        ax1.coastlines()
        plt.colorbar()
        plt.show()
        print(a)

In [ ]:
print(entries)